In [17]:
import networkx as nx
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter
import itertools
import requests
from io import StringIO
import numpy as np

In [18]:
countries = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia",
    "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium",
    "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria",
    "Burkina Faso", "Burundi", "Cambodia", "Cameroon", "Canada", "Cape Verde", "Central African Republic", "Chad",
    "Chile", "China", "Colombia", "Comoros", "Congo", "Costa Rica", "Côte d'Ivoire", "Croatia", "Cuba", "Cyprus",
    "Czech Republic", "North Korea", "Democratic Republic of the Congo", "Denmark", "Djibouti", "Dominica",
    "Dominican Republic", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Eswatini",
    "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada",
    "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Honduras", "Hungary", "Iceland", "India",
    "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya",
    "Kiribati", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein",
    "Lithuania", "Luxembourg", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands",
    "Mauritania", "Mauritius", "Mexico", "Micronesia", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique",
    "Myanmar", "Namibia", "Nauru", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria",
    "North Macedonia", "Norway", "Oman", "Pakistan", "Palau", "Panama", "Papua New Guinea", "Paraguay", "Peru",
    "Philippines", "Poland", "Portugal", "Qatar", "South Korea", "Moldova", "Romania", "Russia", "Rwanda",
    "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", "Samoa", "San Marino",
    "São Tomé and Príncipe", "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore",
    "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Spain", "Sri Lanka",
    "Sudan", "Suriname", "Sweden", "Switzerland", "Syria", "Tajikistan", "Tanzania", "Thailand", "Timor-Leste",
    "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine",
    "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Venezuela",
    "Vietnam", "Yemen", "Zambia", "Zimbabwe"
]
print(f"Loaded {len(countries)} countries.")

Loaded 193 countries.


In [19]:
country_graph = nx.DiGraph()
country_graph.add_nodes_from(countries)
country_graph.add_edges_from([(c1, c2) for c1, c2 in itertools.product(countries, repeat=2) if c1 != c2 and c1.lower()[-1] == c2.lower()[0]])

print(f"Country graph created with {country_graph.number_of_nodes()} nodes and {country_graph.number_of_edges()} edges.")

Country graph created with 193 nodes and 2014 edges.


In [28]:
def create_clean_interactive_graph(G):
    pos = nx.kamada_kawai_layout(G)

    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.4, color='#888'),
        hoverinfo='none',
        mode='lines',
        visible='legendonly',
        name='Connections'
    )

    node_x, node_y, node_hovertext, node_color, node_size = [], [], [], [], []
    for node in G.nodes():
        x, y = pos[node]
        in_degree = G.in_degree(node)
        out_degree = G.out_degree(node)

        node_x.append(x)
        node_y.append(y)

        hovertext = (
            f"{node}"
            f"Options From Here: {out_degree}"
            f"Paths To Here: {in_degree}"
            f"Strategic Value: {out_degree - in_degree}"
        )
        node_hovertext.append(hovertext)
        node_color.append(out_degree - in_degree)
        node_size.append(5 + (in_degree + out_degree) / 2)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        text=node_hovertext,
        marker=dict(
            showscale=True,
            colorscale='RdYlGn',

            color=node_color,
            size=node_size,
            colorbar=dict(
                thickness=15,
                title='Strategic Value',
                xanchor='left',
                titleside='right'
            ),
            line_width=1,
            line_color='black'
        ),
        name='Countries'
    )

    fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Atlas Game: Strategic Map of Countries',
                titlefont_size=20,
                showlegend=True,
                legend_title_text='Toggle View',
                legend=dict(
                    x=0.01,
                    y=0.99,
                    xanchor='left',
                    yanchor='top'
                ),
                hovermode='closest',
                margin=dict(b=5, l=5, r=5, t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                plot_bgcolor='white',
                height=700,
                width=1000
            )
    )

    fig.show()

create_clean_interactive_graph(country_graph)



In [25]:
def generate_graph_report(G, name):
    print("\n" + "="*70)
    print(f" COMPREHENSIVE STRATEGIC ANALYSIS: {name.upper()} GRAPH")
    print("="*70)

    #Graph Density
    density = nx.density(G)
    print(f"\n 1: Basic Structure & Density")
    print(f"Nodes: {G.number_of_nodes()} | Edges: {G.number_of_edges()}")
    print(f"Graph Density: {density:.4f}")

    #Degree Centrality & Terminal Nodes
    in_degrees = dict(G.in_degree)
    out_degrees = dict(G.out_degree)
    dead_ends = [node for node, degree in out_degrees.items() if degree == 0]
    top_in = sorted(in_degrees.items(), key=lambda x: x[1], reverse=True)[:5]
    top_out = sorted(out_degrees.items(), key=lambda x: x[1], reverse=True)[:5]

    print(f"\n 2: Degree Centrality & Traps")
    print(f"Most Common Destinations (Top 5 by In-Degree):")
    for node, degree in top_in: print(f"  - {node}: {degree} incoming edges")
    print(f"Most Powerful Moves (Top 5 by Out-Degree):")
    for node, degree in top_out: print(f"  - {node}: {degree} outgoing options")
    print(f"Identified Terminal Nodes ('Dead Ends'): {len(dead_ends)}")
    if dead_ends: print(f"  Examples: {', '.join(dead_ends[:5])}")

    #Betweenness Centrality
    betweenness = nx.betweenness_centrality(G)
    top_betweenness = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)[:5]
    print(f"\n 3: Betweenness Centrality - Strategic Bridges ")
    for node, score in top_betweenness: print(f"  - {node}: {score:.4f}")

    #PageRank Centrality
    pagerank = nx.pagerank(G)
    top_pagerank = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:5]
    print(f" 4: PageRank - Influential Hubs ")
    for node, score in top_pagerank: print(f"  - {node}: {score:.4f}")

    #Strongly Connected Components (SCCs)
    sccs = list(nx.strongly_connected_components(G))
    largest_scc = max(sccs, key=len) if sccs else []
    print(f"\n 5: Structural Cohesion - The Game's Core ")
    print(f"Number of Strongly Connected Components: {len(sccs)}")
    print(f"Size of Largest SCC ('The Core Game'): {len(largest_scc)} nodes ({len(largest_scc)/G.number_of_nodes():.1%})")

    #Letter Economy Analysis
    start_letters = Counter(c[0].lower() for c in G.nodes())
    end_letters = Counter(c[-1].lower() for c in G.nodes())
    all_letters = sorted(list(set(start_letters.keys()) | set(end_letters.keys())))
    advantage = {l: start_letters.get(l, 0) - end_letters.get(l, 0) for l in all_letters}
    best_letters = sorted(advantage.items(), key=lambda x: x[1], reverse=True)[:5]
    worst_letters = sorted(advantage.items(), key=lambda x: x[1])[:5]
    print(f"\n 6: Letter Economy - Strategic Resources ")
    print("Safest Letters to End On (Highest Advantage):")
    for letter, score in best_letters: print(f"  - '{letter.upper()}': +{score} advantage")
    print("Most Risky Letters to End On (Highest Disadvantage):")
    for letter, score in worst_letters: print(f"  - '{letter.upper()}': {score} disadvantage")

    return {
        'in_degrees': in_degrees, 'out_degrees': out_degrees, 'start_letters': start_letters,
        'end_letters': end_letters, 'all_letters': all_letters, 'letter_advantage': advantage
    }

In [21]:
def create_analysis_dashboard(results, name):
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=("Options After Naming (Out-Degree)", "How Often a Place is an Option (In-Degree)",
                        "Starting vs. Ending Letter Frequency", "Strategic Letter Advantage"),
        vertical_spacing=0.15, horizontal_spacing=0.1
    )
    fig.add_trace(go.Histogram(x=list(results['out_degrees'].values()), name='Out-Degree', marker_color='#636EFA'), row=1, col=1)
    fig.add_trace(go.Histogram(x=list(results['in_degrees'].values()), name='In-Degree', marker_color='#EF553B'), row=1, col=2)

    start_counts = [results['start_letters'].get(l, 0) for l in results['all_letters']]
    end_counts = [results['end_letters'].get(l, 0) for l in results['all_letters']]
    fig.add_trace(go.Bar(x=[l.upper() for l in results['all_letters']], y=start_counts, name='Starts With'), row=2, col=1)
    fig.add_trace(go.Bar(x=[l.upper() for l in results['all_letters']], y=end_counts, name='Ends With'), row=2, col=1)

    adv = results['letter_advantage']
    letters = sorted(adv.keys())
    advantages = [adv[l] for l in letters]
    colors = ['#00CC96' if val > 0 else '#FFA15A' for val in advantages]
    fig.add_trace(go.Bar(x=[l.upper() for l in letters], y=advantages, marker_color=colors, name='Advantage'), row=2, col=2)

    fig.update_layout(height=800, width=1200, title_text=f"Analysis Dashboard for {name} Graph",
                      barmode='group', showlegend=False)
    fig.update_xaxes(title_text="Degree Count", row=1, col=1); fig.update_xaxes(title_text="Degree Count", row=1, col=2)
    fig.update_xaxes(title_text="Letter", row=2, col=1); fig.update_xaxes(title_text="Letter", row=2, col=2)
    fig.show()

In [22]:
class AtlasGameAdvisor:
    def __init__(self, graph, analysis_results):
        self.graph = graph
        self.analysis = analysis_results
        self.used_items = set()
        self.item_scores = self._calculate_scores()

    def _calculate_scores(self):
        scores = {}
        out_degrees = self.analysis['out_degrees']
        pagerank = self.analysis.get('pagerank', {})
        betweenness = self.analysis.get('betweenness', {})

        if not pagerank: pagerank = nx.pagerank(self.graph)
        if not betweenness: betweenness = nx.betweenness_centrality(self.graph)

        for item in self.graph.nodes():
            out_deg = out_degrees.get(item, 0)
            pr = pagerank.get(item, 0)
            b = betweenness.get(item, 0)
            score = (out_deg * 2) + (pr * 500) + (b * 200)
            if out_deg == 0:
                score -= 100
            scores[item] = score
        return scores

    def recommend_moves(self, last_item):
        valid_moves = [s for s in self.graph.successors(last_item) if s not in self.used_items]
        if not valid_moves:
            return "Opponent is trapped! You win!"
        return sorted([(move, self.item_scores.get(move, 0)) for move in valid_moves], key=lambda x: x[1], reverse=True)

    def mark_used(self, item):
        self.used_items.add(item)

    def reset(self):
        self.used_items.clear()

def start_interactive_session(advisor, graph_nodes):
    print("\n" + "="*60 + "\nATLAS GAME STRATEGY ADVISOR\n" + "="*60)
    print("Enter opponent's move to get strategic advice. Commands: 'reset', 'used', 'quit'")

    while True:
        user_input = input("\n> Opponent played: ").strip()

        if user_input.lower() == 'quit': break
        if user_input.lower() == 'reset':
            advisor.reset(); print("Game has been reset."); continue
        if user_input.lower() == 'used':
            print("Used:", sorted(list(advisor.used_items)) if advisor.used_items else "None"); continue

        matches = [c for c in graph_nodes if c.lower().startswith(user_input.lower())]
        if not matches:
            print(f"Sorry, couldn't find '{user_input}'."); continue

        selected_item = matches[0]
        advisor.mark_used(selected_item)
        print(f"Opponent's move '{selected_item}' marked as used.")

        recommendations = advisor.recommend_moves(selected_item)

        if isinstance(recommendations, str):
            print(f"\n{recommendations}"); break

        print("\n--- Recommended Moves (Best to Worst) ---")
        for i, (move, score) in enumerate(recommendations[:5]):
            dead_end_tag = " (WINNING TRAP!)" if advisor.analysis['out_degrees'].get(move, 0) == 0 else ""
            print(f"{i+1}. {move}{dead_end_tag}")

        if recommendations:
            best_move = recommendations[0][0]
            choice = input(f"Execute best move '{best_move}'? (y/n): ").strip().lower()
            if choice == 'y':
                advisor.mark_used(best_move)
                print(f"'{best_move}' marked as used. Your turn is over.")

In [26]:
#text report
analysis_results = generate_graph_report(country_graph, "Country")

#visual dashboard
create_analysis_dashboard(analysis_results, "Country")

analysis_results['pagerank'] = nx.pagerank(country_graph)
analysis_results['betweenness'] = nx.betweenness_centrality(country_graph)
country_advisor = AtlasGameAdvisor(country_graph, analysis_results)



 COMPREHENSIVE STRATEGIC ANALYSIS: COUNTRY GRAPH

 1: Basic Structure & Density
Nodes: 193 | Edges: 2014
Graph Density: 0.0544

 2: Degree Centrality & Traps
Most Common Destinations (Top 5 by In-Degree):
  - Afghanistan: 73 incoming edges
  - Azerbaijan: 73 incoming edges
  - Albania: 72 incoming edges
  - Algeria: 72 incoming edges
  - Andorra: 72 incoming edges
Most Powerful Moves (Top 5 by Out-Degree):
  - Bahamas: 26 outgoing options
  - Barbados: 26 outgoing options
  - Belarus: 26 outgoing options
  - Comoros: 26 outgoing options
  - Cyprus: 26 outgoing options
Identified Terminal Nodes ('Dead Ends'): 0

 3: Betweenness Centrality - Strategic Bridges 
  - Afghanistan: 0.1594
  - Azerbaijan: 0.1594
  - Netherlands: 0.1455
  - New Zealand: 0.1314
  - Nauru: 0.1232
 4: PageRank - Influential Hubs 
  - Afghanistan: 0.0505
  - Azerbaijan: 0.0505
  - Albania: 0.0466
  - Algeria: 0.0466
  - Andorra: 0.0466

 5: Structural Cohesion - The Game's Core 
Number of Strongly Connected Compon

In [24]:
start_interactive_session(country_advisor, list(country_graph.nodes()))


ATLAS GAME STRATEGY ADVISOR
Enter opponent's move to get strategic advice. Commands: 'reset', 'used', 'quit'

> Opponent played: Nepal
Opponent's move 'Nepal' marked as used.

--- Recommended Moves (Best to Worst) ---
1. Laos
2. Luxembourg
3. Lebanon
4. Liechtenstein
5. Latvia
Execute best move 'Laos'? (y/n): y
'Laos' marked as used. Your turn is over.

> Opponent played: Singapore
Opponent's move 'Singapore' marked as used.

--- Recommended Moves (Best to Worst) ---
1. Egypt
2. Equatorial Guinea
3. Eritrea
4. Estonia
5. Ethiopia
Execute best move 'Egypt'? (y/n): y
'Egypt' marked as used. Your turn is over.

> Opponent played: Tanzania
Opponent's move 'Tanzania' marked as used.

--- Recommended Moves (Best to Worst) ---
1. Afghanistan
2. Azerbaijan
3. Albania
4. Algeria
5. Andorra
Execute best move 'Afghanistan'? (y/n): y
'Afghanistan' marked as used. Your turn is over.

> Opponent played: Norway
Opponent's move 'Norway' marked as used.

--- Recommended Moves (Best to Worst) ---
1. Ye